In [1]:
import os
from glob import glob
import subprocess
import nibabel as nib
from dipy.align.reslice import reslice

# dcm2niix -z y -m y -o ~/Batch/niftix '/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD'
# med2image -i /home/rodrigo/Batch/niftix/test99-5mm.nii.gz -d /home/rodrigo/Batch/niftix/ -o image.png -s -1

# src = '/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD'
# out = '/home/rodrigo/Batch/nifti8/99-ct-pre-contrast-5mm-std.nii.gz'
# dicom2nifti.dicom_series_to_nifti(src, out, reorient_nifti=True)

/home/rodrigo/miniconda3/envs/pydicomenv/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [23]:
# Create 3 functions:
# 1) Convert dicom chosen serie to Nifti
# 2) Reslice Nifti to 5mm
# 3) Generate Pngs from a Nifti Volume
filepath = '/home/rodrigo/Batch/ToReslice.txt'

In [24]:
# TODO: move this to nonfunction
def dicoms_to_raw(path):
    cmd = "ok"
    for dcm in glob(path.strip('\n') + '/*.dcm'):
        cmd = "gdcmconv --raw '" + dcm + "' '" + dcm + "'"
        result = subprocess.call(cmd, shell=True)
    print(cmd)

def convert_series_to_nifti(filepath):
    # serie: /home/rodrigo/Batch/CQ500CT100/serie_name
    for serie in open(filepath, 'r').readlines():
        print(serie)
        # TODO: move this to nonfunction
        dicoms_to_raw(serie)
        # nii_name: CQ500CT100-serie_name
        nii_name = serie.split('/')[-2] + '_%p'
        print(nii_name)
        # nii_dir: /home/rodrigo/Batch/nifti/
        nii_dir = os.path.dirname(filepath) + '/nifti/' 
        os.makedirs(nii_dir, exist_ok=True)
        serie = serie.strip('\n')
        cmd = "dcm2niix -z y -m y -f " + nii_name + " -o " + nii_dir + " '" + serie + "'"
        print(cmd)
        result = subprocess.call(cmd, shell=True)
        print(nii_name + " result: " + str(result))
    return nii_dir
        
def reslice_nifti(nii_dir):
    for nii in os.listdir(nii_dir):
        fimg = nii_dir + nii
        print("fimg")
        print(fimg)
        img = nib.load(fimg)
        data = img.get_data()
        # Reslice
        new_zooms = (1., 1., 5.)
        affine = img.affine
        zooms = img.header.get_zooms()[:3]
        data2, affine2 = reslice(data, affine, zooms, new_zooms)  
        # Save File
        img2 = nib.Nifti1Image(data2, affine2)
        new_name = fimg.strip('.nii.gz') + '_5mm.nii.gz'
        nib.save(img2, new_name)

def generate_pngs(nii_dir):
    for nii in glob(nii_dir + "/*_5mm.nii.gz"):
        exam = nii.split("/")[5].split("_")[0]
        png_dir = nii_dir + exam
        os.makedirs(png_dir, exist_ok=True)
        cmd = 'med2image -i ' + nii + ' -d ' + png_dir + ' -o ' + exam + '.png -s -1'
        subprocess.call(cmd, shell=True)

            
    


In [25]:
nii_dir = convert_series_to_nifti(filepath)
reslice_nifti(nii_dir)
generate_pngs(nii_dir)

/home/rodrigo/Batch/CQ500CT0/CT PLAIN THIN

gdcmconv --raw '/home/rodrigo/Batch/CQ500CT0/CT PLAIN THIN/CT000191.dcm' '/home/rodrigo/Batch/CQ500CT0/CT PLAIN THIN/CT000191.dcm'
CQ500CT0_%p
dcm2niix -z y -m y -f CQ500CT0_%p -o /home/rodrigo/Batch/nifti/ '/home/rodrigo/Batch/CQ500CT0/CT PLAIN THIN'
CQ500CT0_%p result: 0
/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD

gdcmconv --raw '/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD/CT000018.dcm' '/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD/CT000018.dcm'
CQ500CT99_%p
dcm2niix -z y -m y -f CQ500CT99_%p -o /home/rodrigo/Batch/nifti/ '/home/rodrigo/Batch/CQ500CT99/CT PRE CONTRAST 5MM STD'
CQ500CT99_%p result: 0
/home/rodrigo/Batch/CQ500CT0/CT PLAIN

ok
CQ500CT0_%p
dcm2niix -z y -m y -f CQ500CT0_%p -o /home/rodrigo/Batch/nifti/ '/home/rodrigo/Batch/CQ500CT0/CT PLAIN'
CQ500CT0_%p result: 1
fimg
/home/rodrigo/Batch/nifti/CQ500CT0_PLAIN_THIN_Tilt_1.nii.gz
fimg
/home/rodrigo/Batch/nifti/CQ500CT99_PRE_CONTRAST_5MM_STD.nii.gz
fim